# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 statistics.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets 

In [ ]:
%sx ./refreshdata local patch

---
## Confirmed, deaths, recovered datasets

In [ ]:
import os

import numpy as np
import pandas as pd

from covidvu import utils
from covidvu.cryostation import Cryostation


pd.options.mode.chained_assignment = None

In [ ]:
databasePath = './database/virustrack.db'
with Cryostation(databasePath=databasePath) as storage:
    confirmedCases = storage.timeSeriesFor(regionType = 'country', casesType = 'confirmed', disableProgressBar=False)

In [ ]:
ROLLING_WINDOW = 7
MIN_CASES = 100

In [ ]:
confirmedCasesSmooth = (confirmedCases+1).rolling(ROLLING_WINDOW, axis=0).mean()

In [ ]:
confirmedCasesSmooth[confirmedCasesSmooth < MIN_CASES] = np.nan

In [ ]:
newCases = confirmedCasesSmooth.diff(axis=0)
growthFactor = newCases/newCases.shift(1)

---
## Interactive plots

In [ ]:
from covidvu import visualize
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed

In [ ]:
countries = list(confirmedCases.columns)
multiCountry = widgets.SelectMultiple(
    options=countries,
    value=['US'],
    description='Country',
    disabled=False
)
log = widgets.Checkbox(value=False, description='Log scale')

### Growth factor

When the growth factor is consistently below 1, we have moved to the saturation phase.

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(growthFactor), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Growth factor'),
         title=fixed(r'COVID-19 growth factor (new cases today)/(new cases yesterday)')
);

## Gauge charts

In [ ]:
from covidvu.pipeline.vugrowth import computeGrowth
import plotly.graph_objects as go

In [ ]:
computeGrowth(regionType = 'country', casesType = 'confirmed',               
              todayDate = pd.to_datetime('2020-04-16').date(),
              disableProgressBar=False,
             )

In [ ]:
countryName = 'US'

In [ ]:
with Cryostation(databasePath=databasePath) as storage:
    growthFactorUS = storage[countryName]['growth']

In [ ]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = growthFactorUS['yesterday'],
    mode = "gauge+number+delta",
    title = {'text': f"Growth factor: yesterday in {countryName}"},
    delta = {'reference': growthFactorUS['lastWeek'],
             'valueformat':".2f",
             'decreasing':{'color':'#5dc724'},
             'increasing':{'color':'#ab0a0a'},
            },
    gauge = {
             'steps' : [
                 {'range': [0, 0.9], 'color': "#5dc724"},
                 {'range': [0.9, 1], 'color': "#eba417"},
                 {'range': [1, 3], 'color': "#ab0a0a"},
             ],
             'bar': {'color': "#7e7c8a"},
             'axis': {'range': [None, 2],
                      'tickmode': 'array', 'tickvals':[0.45, 0.95, 1.5], 'ticktext':['Flattening', 
                                                                                   'Not flat enough', 
                                                                                   'Growing'], 
                      'tickfont':{'size':20},
                      
                     },
            },


)
               )

fig.show()

In [ ]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = growthFactorUS['lastWeek'],
    mode = "gauge+number+delta",
    title = {'text': f"Growth factor: last week in {countryName}"},
    delta = {'reference': growthFactorUS['lastTwoWeek'],
             'valueformat':".2f",
             'decreasing':{'color':'#5dc724'},
             'increasing':{'color':'#ab0a0a'},
            },
    gauge = {
             'steps' : [
                 {'range': [0, 0.9], 'color': "#5dc724"},
                 {'range': [0.9, 1], 'color': "#eba417"},
                 {'range': [1, 3], 'color': "#ab0a0a"},
             ],
             'bar': {'color': "#7e7c8a"},
             'axis': {'range': [None, 2],
                      'tickmode': 'array', 'tickvals':[0.45, 0.95, 1.5], 'ticktext':['Flattening', 
                                                                                   'Not flat enough', 
                                                                                   'Growing'], 
                      'tickfont':{'size':20},
                      
                     },
            },


)
               )

fig.show()

---
&#169; the COVIDvu Contributors.  All rights reserved.